In [1]:
import pandas as pd
import numpy as np

In [2]:
movies = pd.read_csv('tmdb_5000_movies.csv')
credits = pd.read_csv('tmdb_5000_credits.csv')

In [14]:
def valuta_chiave(df, colonna):
    return len(df[colonna]) == len(df[colonna].drop_duplicates())

In [20]:
valuta_chiave(movies, 'id')

True

In [17]:
valuta_chiave(credits, 'movie_id')

True

In [21]:
def normalizza(movie_id, colonna, dataset):
    norm = dataset[colonna].apply(pd.read_json)
    movie = dataset[movie_id]
    for num in range(len(dataset)):
        norm[num][movie_id] = movie[num]
    return pd.concat(norm.values).reset_index().drop('index', axis = 1)

1) For each movie, compute the number of cast members

In [27]:
cast = normalizza('movie_id', 'cast', credits)
cast_number = cast.groupby('movie_id').size()
cast_number = cast_number.to_frame().rename(columns= {0: 'cast_number'}).reset_index()
cast_numbers = pd.merge(left = cast_number, right = credits[['movie_id', 'title']], on = 'movie_id')

2) How many movies do not have a homepage?

In [28]:
len(movies['homepage']) - len(movies['homepage'].dropna())

3091

3) For each year, how many movies do not have a homepage?

In [46]:
moviey = movies[movies['homepage'].isnull()]
moviey['year'] = movies['release_date'].str.extract('(.*)-(.*)-(.*)')[0]
moviey = moviey.groupby('year').size().to_frame().rename(columns = {0 : 'number'}).reset_index()

C:\Users\aless\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: FutureWarning: currently extract(expand=None) means expand=False (return Index/Series/DataFrame) but in a future version of pandas this will be changed to expand=True (return DataFrame)
  
C:\Users\aless\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


4) Extract the domain of each homepage.

In [48]:
import tldextract
movie_d = movies['homepage'].dropna()
movie_d = movie_d.to_frame()
movie_d['indice'] = -1
salva_doppi = pd.DataFrame()
salva_doppi['index'] = 0
conta = 0
for riga in movie_d.index:
    url = movie_d['homepage'][riga]
    url = url.split()
    if(len(url) != 1):
        movie_d['indice'][riga] = riga
        salva_doppi = salva_doppi.append(url)
        for split in range(conta,len(url)):
            salva_doppi['index'][split] = riga
        conta = conta + len(url)
salva_doppi = salva_doppi.rename(columns = {0: 'homepage'})
movie_d = movie_d.drop(movie_d[movie_d['indice'] != -1].index)
movie_d = movie_d.reset_index()
movie_d = movie_d.drop('indice', axis = 1)
movie_d = movie_d.append(salva_doppi)
movie_d = movie_d.reset_index().drop('level_0', axis = 1)
movie_d['domain'] = 0
for riga in movie_d.index:
    url = movie_d['homepage'][riga]
    movie_d['domain'][riga] = tldextract.extract(url).suffix
movie_t = movies['original_title']
movie_t = movie_t.to_frame()
movie_t = movie_t.reset_index()
movie_d = pd.merge(left = movie_d, right = movie_t, on = 'index')
movie_d = movie_d.set_index('index')

C:\Users\aless\Anaconda3\lib\site-packages\ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if sys.path[0] == '':
C:\Users\aless\Anaconda3\lib\site-packages\ipykernel_launcher.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app
C:\Users\aless\Anaconda3\lib\site-packages\ipykernel_launcher.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Users\aless\Anaconda3\lib\site-packages\pandas\core\indexing.py:179: SettingWithCop

5) Extract a set of normalized tables. That is, each entry of a normalized table must contain exactly one value (not a list or a dictionary)

In [52]:
movies_n = movies.drop(['genres', 'keywords', 'production_companies', 'production_countries', 'spoken_languages'], axis = 1)
credits_n = credits.drop(['cast', 'crew'], axis = 1)
genres = normalizza('id','genres', movies)
keywords = normalizza('id', 'keywords', movies)
production_companies = normalizza('id','production_companies', movies)
production_countries = normalizza('id', 'production_countries', movies).rename(columns = {'iso_3166_1':'country_id'})
spoken_languages = normalizza('id', 'spoken_languages', movies).rename(columns = {'iso_639_1':'language_id'})
crew = normalizza('movie_id','crew', credits)

6) For each movie, compute the gross margin (difference between revenue and budget)


In [53]:
movies_n['gross_margin'] = movies_n['revenue'] - movies_n['budget']

7) For each movie, compute the number of crew members

In [73]:
crew_n = pd.merge(left = crew, right = credits_n, on = 'movie_id')
crew_number = crew_n.groupby(['movie_id', 'title']).count()['credit_id'].to_frame().reset_index().rename(
                                                                                        columns = {'credit_id' : 'crew_number'})

8) For each movie, compute the number of directors

In [78]:
directors = (crew_n[crew_n['job'] == 'Director']).groupby(['movie_id', 'title']).size().to_frame().rename(
                                                                                columns = {0 : 'number'}).reset_index()

9) For each language, compute the number of movies where such language is spoken.

In [84]:
movies_language = spoken_languages.groupby(['language_id', 'name']).size().to_frame().rename(
                                                                    columns = {0 : 'number'}).reset_index()

10) For each company and each decade, compute the overall revenue

In [90]:
movies_n['year'] = movies_n['release_date'].str.extract('(.*)-(.*)-(.*)')[0]
movies_n['decade'] = (movies_n['year'].str.extract('(\d\d\d)')) + '0'
movies_companies = movies_n.merge(production_companies, on = 'id')
overall_revenue = movies_companies.groupby(['name', 'decade']).sum()['revenue'].to_frame().rename(
                                                            columns = {'revenue' : 'overall_revenue'}).reset_index()

C:\Users\aless\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: currently extract(expand=None) means expand=False (return Index/Series/DataFrame) but in a future version of pandas this will be changed to expand=True (return DataFrame)
  """Entry point for launching an IPython kernel.
C:\Users\aless\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: FutureWarning: currently extract(expand=None) means expand=False (return Index/Series/DataFrame) but in a future version of pandas this will be changed to expand=True (return DataFrame)
  


11) For each decade, compute the company with maximum revenue

In [92]:
maximum_revenue = movies_companies.groupby(['decade']).max()['revenue'].to_frame()
max_revenue = maximum_revenue.merge(movies_companies, on = 'revenue')
max_revenues = max_revenue[['revenue', 'decade', 'name']]

12) In each year, how many movies have revenue smaller than the budget?

In [98]:
movies_sm = (movies_n[movies_n['gross_margin'] < 0]).groupby('year').size().to_frame().rename(
                                                                                columns = {0 : 'number'}).reset_index()

13) Distribute the revenue according to the order of appearance in a movie. Assume that the i-th actor contributes twice as much as the (i+1)-th actor to the revenue.

In [104]:
import math
cast_ = pd.merge(left = cast, right = credits[['movie_id', 'title']], on = 'movie_id')
cast_ = cast_[['name', 'order', 'title']]
movies_ = movies_n[['original_title', 'revenue']]
cast_movies = pd.merge(left = cast_, right = movies_, left_on = 'title', right_on = 'original_title').drop('original_title',
                                                                                                          axis = 1)
number_actor = cast_movies.groupby('title').size().to_frame().reset_index().rename(columns = {0: 'number_actor'})
cast_movies = pd.merge(left = cast_movies, right = number_actor, on = 'title')
massimo = cast_movies.groupby('title').max()['order'].to_frame().reset_index().rename(columns = {'order' : 'max_order'})
cast_movies = pd.merge(left = cast_movies, right = massimo, on = 'title')
cast_movies['ordine'] = cast_movies['number_actor'] - cast_movies['max_order'] + cast_movies['order']
cast_movies = cast_movies.drop('order', axis = 1).rename(columns = {'ordine' : 'order'}).drop('max_order', axis = 1)
def exp(x):
    return math.pow(2, x)
cast_movies['exp'] = cast_movies['number_actor'].apply(exp)
cast_movies['exp'] = cast_movies['exp'] - 1 
cast_movies['part'] = cast_movies['revenue'] / cast_movies['exp']
ordine = {}
index = 0
for titoli in cast_movies['title'].unique():
    massimo = len(cast_movies[cast_movies['title'] == titoli])
    for numero in range(0, massimo):
        ordine[index] = massimo - numero - 1
        index = index + 1
ordine = pd.DataFrame.from_dict(ordine, orient = 'index')
ordine = ordine.rename(columns = {0 : 'c_order'})
cast_movies['c_order'] = ordine['c_order']
cast_movies['order'] = cast_movies['number_actor'] - cast_movies['c_order']
def fattore(x):
    return math.pow(2, x)
cast_movies['fattore'] = cast_movies['c_order'].apply(fattore)
cast_movies['revenue_part'] = cast_movies['fattore'] * cast_movies['part']
cast_movies = cast_movies[['name', 'title', 'order', 'revenue', 'revenue_part']]
somma = cast_movies.groupby('title').sum()['revenue_part'].reset_index().rename(columns = {'revenue_part' : 'revenue_sum'})
cast_movies = pd.merge(left = cast_movies, right = somma, on = 'title')
def to_integer(x):
    return int(x)
cast_movies['revenue_part'] = cast_movies['revenue_part'].apply(to_integer)
cast_movies['revenue_sum'] = cast_movies['revenue_sum'].apply(to_integer)

14) For each actor find the total revenue attributed to him/her.

In [106]:
actor_revenue = cast_movies.groupby('name').sum()['revenue_part'].to_frame().rename(columns = {'revenue_part' :
                                                                                              'total_revenue'}).reset_index()

15) Find the actor that is responsible for the most overall revenue.

In [108]:
actor_revenue[actor_revenue['total_revenue'] == np.max(actor_revenue['total_revenue'])]

,name,total_revenue
48131,Tom Cruise,3976119747
